## Twitch Clip AutoGenerator
## By: Peter Larcheveque

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import ast
import json
import new_token
import os
import matplotlib.pyplot as plt
import seaborn as sns
import imageio
import subprocess
from subprocess import *
import regex as re
import time

from subprocess import Popen, PIPE, STDOUT
from io import StringIO

# file editing imports:
from io import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
from moviepy.audio import *
from moviepy.video.fx.all import resize
import assemblyai as aai
#from scipy.io import wavfile
#from faster_whisper import WhisperModel
import ffmpeg
import glob

## Obtain access token

In [3]:
t = new_token.token_class()

In [4]:
client_id, client_secret = t.get_info()

In [5]:
access_credentials = t.get_access_credentials()

## Use access token to request server data from twitch -- get 20 most active streams

In [5]:
url = 'https://api.twitch.tv/helix/streams?language=en' # + '?' + 'user_id=123456&type=highlight'
print(url)

https://api.twitch.tv/helix/streams?language=en


In [6]:
r = requests.get(url, headers = access_credentials)
#r.json()['data']
r

<Response [200]>

In [7]:
## get top 20 streamers and put their data into a df
top_20_streamers = json.loads(r.content)['data']

master_data = []
for i in range(len(top_20_streamers)):
    data = []
    for key in top_20_streamers[i].keys():
        data.append(top_20_streamers[i][key])
    master_data.append(data)
        
top_20_df = pd.DataFrame(data = master_data, columns = top_20_streamers[i].keys())

In [8]:
top_20_df.head()

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,tags,is_mature
0,50727633133,552120296,zackrawrr,zackrawrr,435870350,Dragon's Dogma II,live,NEWS/DRAMA/REACTS/DRAGONS DOGMA 2 ENDGAME @Sta...,16937,2024-03-26T04:08:02Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[English, OTK, otknetwork, StarForgePCs, wow, ...",False
1,50726201597,175831187,ironmouse,ironmouse,252812003,Supermarket Simulator,live,PLAYING WITH MY FOOD NEED TO READ 2k BEFORE 4 ...,8208,2024-03-26T00:56:52Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[Anime, Singing, PuertoRico, vshojo, Latino, v...",True
2,50726361149,38881685,boxbox,boxbox,513143,Teamfight Tactics,live,ok this B patch is my patch !BoxBoxBootcamp,7296,2024-03-26T01:17:39Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[Pog, FLUSH, Madge, Tall, eatsannoyingviewers,...",False
3,43899949307,151368796,piratesoftware,PirateSoftware,1469308723,Software and Game Development,live,GAME DEV Q/A 💜 Go Make Games 💛 @FerretSoftware...,6563,2024-03-26T07:09:01Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[Chatting, Memes, English, NotMyBirthday, Game...",False
4,42137322968,540056482,shanks_ttv,shanks_ttv,516575,VALORANT,live,"HANDSOME, BEAUTIFUL, TALL, INTELLIGENT VIETNAM...",5636,2024-03-26T01:10:09Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[English],True


## Get videos from top 20 streamers

In [9]:
streamer_id = top_20_df.iloc[0,:]['user_id']
streamer_id

'552120296'

In [ ]:
url = 'https://api.twitch.tv/helix/videos?user_id=' + streamer_id

r = requests.get(url, headers = access_credentials)
json.loads(r.content)

## Terminal Commands to Download the chat/vods
#### documentation: https://pypi.org/project/chat-downloader/
#### documentation: https://pypi.org/project/twitch-dl/

In [6]:
def run(command, print_output=True):
    '''
    Runs terminal commands using subprocess
    command: list - command to be interpreted by OS
    '''
    
    # grab output filename
    output_fn = ''
    popen = Popen(command, stdout=PIPE, stderr=STDOUT, universal_newlines=True)
    out = StringIO()
    for line in popen.stdout:
        if print_output:
            if '/Users/peter/Twitch' in line:
                output_fn = line
            else:
                print(line, end='')
        else:
            out.write(line)

    
    popen.stdout.close()
    return_code = popen.wait()

    if not return_code == 0:
        raise RuntimeError(
            'The process call "{}" returned with code {}. The return code is not 0, thus an error '
            'occurred.'.format(list(command), return_code))

    stdout_string = out.getvalue()
    out.close()

    return stdout_string, output_fn

In [ ]:
# command for downloading twitch vod at specified output location
cmd = ['twitch-dl',
     'download',
     'https://www.twitch.tv/videos/2069606012',
     '-q',
     'source',
     '-w',
     '8',
     '--overwrite',
     '--output',
     '/Users/peter/Twitch/Twitch_VODs/{channel}_{date}_{id}.{format}']
# run command and save fileoutput fiplepath
out = run(cmd)

In [14]:
# use regex to get exact output filepath
vod_output_fn = re.findall('/Users.*.mkv', out[-1])[0]
vod_output_fn

'/Users/peter/Twitch/Twitch_VODs/xQc_2024-02-21_2069606012.mkv'

In [ ]:
# command for downloading twitch vod at specified output location
chat_path = vod_output_fn.replace('mkv','json').replace('Twitch_VODs','Twitch_VODs_Chat')
print(chat_path)
cmd = ['chat_downloader',
     'https://www.twitch.tv/videos/2069606012',
     '--output',
     chat_path]

run(cmd)

## Download Twitch Chat from Channel Clips - Use these chat logs as training data for
## finding clipworthy moments

In [7]:
# grab all occurrences of messages that occur more than 5 times in the clips
keyword_freq = pd.read_csv("corpus.csv")
keyword_freq

,message,proportion
0,KEKW,0.059723
1,LUL,0.025223
2,OMEGALUL,0.024829
3,HUH,0.019827
4,om,0.018523
...,...,...
2138,DinkDonk,0.000076
2139,Gladge,0.000076
2140,Guys?,0.000076
2141,PETTHEMODS,0.000076


## Reading Downloaded VOD Chat

In [199]:
chat = pd.read_json(chat_path)
chat

,author,message,message_id,message_type,time_in_seconds,time_text,timestamp,emotes
0,"{'badges': [{'clickAction': None, 'clickURL': ...",PogU LIVE,cab28d13-f851-4844-a0bb-27a1f5d54a2e,text_message,7,0:07,2024-02-22 03:11:18.646,NaN
1,"{'badges': [{'clickAction': 'VISIT_URL', 'clic...",LIVE,1c1bac51-ed7d-472e-8cb0-82dd96d9a0e4,text_message,8,0:08,2024-02-22 03:11:18.906,NaN
2,"{'badges': [{'clickAction': 'VISIT_URL', 'clic...",LET GO,6c1c5058-0206-4f77-bd2e-1fa9404bdf05,text_message,8,0:08,2024-02-22 03:11:19.286,NaN
3,"{'badges': [{'clickAction': None, 'clickURL': ...",@nadimz222 BAND,e9047586-ce73-4651-990f-e404edbb6b5a,text_message,8,0:08,2024-02-22 03:11:19.535,NaN
4,"{'badges': [{'clickAction': 'VISIT_URL', 'clic...",Pog Pog,d81a3ca4-f45d-4569-8901-1fdeefbafaef,text_message,9,0:09,2024-02-22 03:11:20.139,NaN
...,...,...,...,...,...,...,...,...
106992,"{'badges': [{'clickAction': None, 'clickURL': ...",KICK IS DOWN omE,ba1a27dd-2cc2-45b5-9771-1af748e0726e,text_message,13075,3:37:55,2024-02-22 06:49:05.895,NaN
106993,"{'badges': [{'clickAction': None, 'clickURL': ...",404,56146a63-6967-4aad-ace7-23eb0857ba09,text_message,13075,3:37:55,2024-02-22 06:49:05.964,NaN
106994,"{'badges': [{'clickAction': None, 'clickURL': ...",KICK IS DOWN,bc3efcee-7a6a-45f2-9539-7664281219d1,text_message,13075,3:37:55,2024-02-22 06:49:06.065,NaN
106995,"{'badges': [{'clickAction': None, 'clickURL': ...",404 error Aware,02c5361f-7844-4000-ae0d-35b37c42d734,text_message,13075,3:37:55,2024-02-22 06:49:06.606,NaN


## Method for determining which times to clip in vods
#### 1) Generate weighted corpus based on chat from human created clips
#### 2) Give score to every 30 second window in the stream
#### 3) Grab top percentile windows w/ highest scores

In [8]:
def find_clip_duration(chat_df, keyword_freq):
    '''
    Gives timestamps for start and end duration of Twitch VOD clip.
    
    chat_df: pd.DataFarme - df containing twitch chat
    
    returns: list - contains tuples of form (start_time, end_time)
    '''
    CLIP_DURATION = 30 # MAGIC NUM
    TOP_PERCENTILE = 0.05 # MAGIC NUM
    
    timestamp_dict = dict()
    min_time = min(chat_df['time_in_seconds'])
    max_time = max(chat_df['time_in_seconds'])
    total_time = abs(max_time - min_time)

    windows = total_time // CLIP_DURATION # num windows to loop through

    #NUM_CLIPS = int(np.round(TOP_PERCENTILE * windows))
    NUM_CLIPS = 5
    
    # loop through time windows, calculate score based on how well chat lines up w/ human-generated clips, and 
    # return top percentile timestamps w/ greatest scores
    for window in range(windows):
        score = 0
        temp_df = chat_df[(chat_df['time_in_seconds'] >= (CLIP_DURATION * window)) & \
                        (chat_df['time_in_seconds'] < (CLIP_DURATION * (window + 1)))]
        for idx, elem in temp_df.iterrows():
            if elem['message'] in list(keyword_freq.message.values): # check if chat message is in keyword freq
                freq_score = keyword_freq[keyword_freq['message'] == elem['message']].iloc[0,1]
                score += freq_score

        timestamp_dict[(CLIP_DURATION * window, CLIP_DURATION * (window + 1))] = score #add to dict - timestamp window (key) : score (value)

    # sort dict and return highest score timestamps
    sorted_dict = {k: v for k, v in sorted(timestamp_dict.items(), key=lambda item: item[1], reverse=True)}
    n_items = {k: sorted_dict[k] for k in list(sorted_dict)[:NUM_CLIPS]} # take first NUM_CLIPS items
    timestamps = list(n_items.keys())
    return timestamps

In [201]:
clip_durations = find_clip_duration(chat, keyword_freq)
clip_durations

[(2580, 2610), (8580, 8610), (6990, 7020), (7500, 7530), (8820, 8850)]

## CLIP PIPELINE - HELPER FUNCTIONS

In [9]:
# clip vod
def clip_twitch(twitch_vod, start_time, end_time):
    '''
    clips twitch vod at given times
    
    twitch_vod: str - fp of downloaded twitch vod
    start_time: int - start time (total num seconds) to start clip at
    end_time: int - end time (total num seconds) to end clip at
    
    returns: str - fp of clipped .mp4 twich vod
    '''
    CLIP_PATH = twitch_vod.replace('Twitch_VODs','Twitch_Clips').replace('mkv','mp4')
    
    ffmpeg_extract_subclip(twitch_vod,
                           start_time, 
                           end_time, 
                           targetname=CLIP_PATH)
    
    return CLIP_PATH

In [21]:
# generate subtitles
def generate_transcript(transcriber, input_fp):
    '''
    Generates subtitle file.
    
    transriber: Trascriber object from assembly ai
    input_fp: str - fp of .mp4/.mkv file
    
    returns: str - output fp of .srt subtitle file
    '''
    
    transcript = transcriber.transcribe(input_fp)
    
    SUB_PATH = vod_output_fn.replace('Twitch_VODs','Twitch_VODs_Transcript').replace('.mkv','.srt')
    
    MAX_CHAR = max([len(x) for x in transcript.text.split()])
    srt = transcript.export_subtitles_srt(chars_per_caption=MAX_CHAR)

    # Save it to a file
    with open(SUB_PATH, "w") as f:
        f.write(srt)
        
    return SUB_PATH

In [11]:
# https://www.bannerbear.com/blog/how-to-add-subtitles-to-a-video-with-ffmpeg-5-different-styles/#:~:text=for%20its%20text.-,How%20to%20Change%20the%20Subtitle%20Style,a%20Video%20File%20Using%20FFmpeg
fontstyles = [     
    'Fontname=Trebuchet MS', # anime style
    'Fontname=Roboto,OutlineColour=&H40000000,BorderStyle=3' #  youtube style
    'Fontname=Consolas,BackColour=&H80000000,Spacing=0.2,Outline=0,Shadow=0.75' # netflix style
    'PrimaryColour=&H03fcff,Italic=1,Spacing=0.8' # vintage yellow style
]

font_locs = [
    'Alignment=1', # bottom right
    'Alignment=2', # bottom middle
    'Alignment=3', # bottom left
    'Alignment=4', # top left
    'Alignment=6', # top middle 
    'Alignment=7', # top right
    'Alignment=9', # left middle 
    'Alignment=10', # center
    'Alignment=11', # right middle
]



def add_subtitle_to_video(soft_subtitle, subtitle_file,  subtitle_language, input_video, output_video_name, 
                          font_style = 'Fontname=Trebuchet MS', font_loc = 'Alignment=10'):

    '''
    Adds subtitle to video
    
    soft_subtitle: Boolean - determines if wants soft subtitles
    subtitle_file: str - subtitle fp, should be a .srt file 
    subtitle_language: str - 'eng' is english
    input_video: str - input video fp
    output_video_name: str - output video fp
    font_style: str - font style
    font_loc: str - location of font
    
    returns: None
    '''
    video_input_stream = ffmpeg.input(input_video)
    subtitle_input_stream = ffmpeg.input(subtitle_file)
    output_video = output_video_name
    subtitle_track_title = subtitle_file.replace(".srt", "")
    

    if soft_subtitle:
        stream = ffmpeg.output(
            video_input_stream, subtitle_input_stream, output_video_name, **{"c": "copy", "c:s": "mov_text"},
            **{"metadata:s:s:0": f"language={subtitle_language}",
            "metadata:s:s:0": f"title={subtitle_track_title}"}
        )
        ffmpeg.run(stream, overwrite_output=True)
    else:
        stream = ffmpeg.output(video_input_stream, output_video_name,
                                
                               vf=f"subtitles={subtitle_file}:force_style='{font_loc},{font_style}'")

        ffmpeg.run(stream, overwrite_output=True)
        
        
    return None

## CLIP PIPELINE

In [23]:
# pipe
def clip_pipeline(twitch_vod, timestamp, output_path, transcriber):
    '''
    Takes vod and clips it using the timestamps given. Will go through the following process:
    1. clip vod using timestamp
    2. Generates subtitles for (1)
    3. Adds subtitles to (2), outputs final result
    
    twitch_vod: str - fp of downloaded twitch vod
    gta_vod: str - fp of downloaded gta vod
    timestamp: list of tuples - tuples in the form (start_time, end_time) --> tells us when to clip vod
    output_path: str - output of final result
    
    returns: None
    '''
    
    start_time = timestamp[0]
    end_time = timestamp[1]
    duration = abs(end_time - start_time)
    CLIP_PATH = clip_twitch(twitch_vod, start_time, end_time)
    #GTA_PATH = clip_GTA(gta_vod, duration)
    #STACKED_PATH = stack_clips(CLIP_PATH, GTA_PATH)    
    SUB_PATH = generate_transcript(transcriber, CLIP_PATH)
    
    add_subtitle_to_video(soft_subtitle = False, subtitle_file = SUB_PATH,
                         subtitle_language = 'eng', input_video = CLIP_PATH,
                         output_video_name = output_path,
                         font_style = 'Fontname=Trebuchet MS')
    return None

In [13]:
#https://www.assemblyai.com/docs/speech-to-text/speech-recognition
aai.settings.api_key = "24f3ca78e4474a638696c8fac2cbf55d"
config = aai.TranscriptionConfig(language_detection=True,
                                disfluencies=True,
                             filter_profanity=False)
transcriber = aai.Transcriber(config = config)

In [19]:
def check_dirs(user_name):
    if os.path.exists(f'/Users/peter/Twitch/Twitch_VODs/{user_name}'):
        pass
    else:
        os.mkdir(f'/Users/peter/Twitch/Twitch_VODs/{user_name}')
        print('Directory is created at: ', f'/Users/peter/Twitch/Twitch_VODs/{user_name}')

    if os.path.exists(f'/Users/peter/Twitch/Twitch_VODs_Chat/{user_name}'):
        pass
    else:
        os.mkdir(f'/Users/peter/Twitch/Twitch_VODs_Chat/{user_name}')
        print('Directory is created at: ', f'/Users/peter/Twitch/Twitch_VODs_Chat/{user_name}')
        
    if os.path.exists(f'/Users/peter/Twitch/Twitch_Final_Result/{user_name}'):
        pass
    else:
        os.mkdir(f'/Users/peter/Twitch/Twitch_Final_Result/{user_name}')
        print('Directory is created at: ', f'/Users/peter/Twitch/Twitch_Final_Result/{user_name}')

    if os.path.exists(f'/Users/peter/Twitch/Twitch_Clips/{user_name}'):
        pass
    else:
        os.mkdir(f'/Users/peter/Twitch/Twitch_Clips/{user_name}')
        print('Directory is created at: ', f'/Users/peter/Twitch/Twitch_Clips/{user_name}')

    if os.path.exists(f'/Users/peter/Twitch/Twitch_VODs_Transcript/{user_name}'):
        pass
    else:
        os.mkdir(f'/Users/peter/Twitch/Twitch_VODs_Transcript/{user_name}')
        print('Directory is created at: ', f'/Users/peter/Twitch/Twitch_VODs_Transcript/{user_name}')

In [ ]:
%%capture
# end to end pipeline
streamer_id = '71092938'
url = 'https://api.twitch.tv/helix/videos?user_id=' + streamer_id
r = requests.get(url, headers = access_credentials)
vods = []
temp = json.loads(r.content)['data']
user_name = temp[0]['user_name']
for elem in temp:
    vods.append(elem['url'])

# create folders if not already exist
check_dirs(user_name)


for vod in vods[:3]:
    print('Processing: ',vod)
    cmd = ['twitch-dl',
     'download',
     vod,
     '-q',
     'source',
     '-w',
     '8',
     '--overwrite',
     '--output',
     f'/Users/peter/Twitch/Twitch_VODs/{user_name}/' + '{date}_{id}.{format}']

    # run command and save fileoutput fiplepath
    out = run(cmd)
    vod_output_fn = re.findall('/Users.*.mkv', out[-1])[0]
    print('Vod_output_fn: ', vod_output_fn)

    # command for downloading twitch vod at specified output location
    chat_path = vod_output_fn.replace('mkv','json').replace('Twitch_VODs','Twitch_VODs_Chat')
    print(chat_path)
    cmd = ['chat_downloader',
         vod,
         '--output',
         chat_path]
    
    run(cmd)
    print('here')
    chat = pd.read_json(chat_path)
    
    # grab clip timestamps
    timestamps = find_clip_duration(chat, keyword_freq)
    print('here2')
    # run clipping pipeline
    for timestamp in timestamps:
        start = time.time()
        
        # run through pipeline
        clip_pipeline(
            twitch_vod = vod_output_fn,
            timestamp = timestamp,
            output_path = vod_output_fn.replace('Twitch_VODs','Twitch_Final_Result').replace('.mkv','_' + str(timestamp) + '.mp4'),
            transcriber = transcriber
        )
        
        end = time.time()
        print(end - start)

In [ ]:
# grab clip timestamps
timestamps = find_clip_duration(chat, keyword_freq)
for timestamp in timestamps:
    start = time.time()
    
    # run through pipeline
    clip_pipeline(
        twitch_vod = vod_output_fn,
        gta_vod = "/Users/peter/Twitch/GTA_Gameplay/GTA_test.mkv",
        timestamp = timestamp,
        output_path = vod_output_fn.replace('Twitch_VODs','Twitch_Final_Result').replace('.mkv','_' + str(timestamp) + '.mp4'),
        transcriber = transcriber
    )
    
    end = time.time()
    print(end - start)

## TikTok and Youtube AutoUploaders

In [182]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager as CM
from pynput.keyboard import Key, Controller
import pyautogui

In [193]:
# avoid bot detection
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")

# Chrome is controlled by automated test software
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# avoiding detection
options.add_argument('--disable-blink-features=AutomationControlled')

# Default User Profile
options.add_argument("--profile-directory=Default")
options.add_argument("--user-data-dir=C:/Users/Admin/AppData/Local/Google/Chrome/User Data")

driver = webdriver.Chrome(options=options)
driver.get('https://www.tiktok.com/login')
ActionChains(driver).key_down(Keys.CONTROL).send_keys(
    '-').key_up(Keys.CONTROL).perform()
ActionChains(driver).key_down(Keys.CONTROL).send_keys(
    '-').key_up(Keys.CONTROL).perform()

# login
try:
    my_element = driver.find_element(By.XPATH,'//*[@id="loginContainer"]/div/div/div/div[3]/div[2]')
    my_element.click()
    time.sleep(3)
    my_element = driver.find_element(By.CLASS_NAME,"tiktok-1mgli76-ALink-StyledLink")
    my_element.click()
    time.sleep(3)
    my_element = driver.find_element(By.CSS_SELECTOR,"input.tiktok-11to27l-InputContainer").send_keys('larchevequepeter@gmail.com')
    my_element = driver.find_element(By.CSS_SELECTOR,"input.tiktok-wv3bkt-InputContainer").send_keys('Peterl99!')
    time.sleep(3)
    my_element = driver.find_element(By.CLASS_NAME,"tiktok-11sviba-Button-StyledButton")
    my_element.click()
except:
    pass

#upload
upload = driver.find_element(By.CLASS_NAME, "css-1sacct4-DivUpload")
upload.click()

time.sleep(3)

# upload file
upload_file = '/Users/peter/Twitch/Twitch_Final_Result/xQc_2024-02-21_2069606012_(8580, 8610).mp4'
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/iframe')))
upload_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[1]/div/div/div/div/div[4]/button').click() #.send_keys(upload_file)
time.sleep(3)

# find and enter tiktok mp4 file
keyboard = Controller()

keyboard.type(upload_file)
keyboard.press(Key.enter)
keyboard.release(Key.enter)
keyboard.press(Key.enter)
keyboard.release(Key.enter)

time.sleep(25)

driver.switch_to.default_content()

# edit caption
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/iframe')))
caption = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div[1]/div[2]/div/div/div/div/div/div')
caption.click()

# clear whatever is in caption
actions = ActionChains(driver)
actions.key_down(Keys.COMMAND).send_keys('a').key_up(Keys. COMMAND).perform()

# enter new caption
CAPTION = 'Twitch AutoUploader Caption Test'
caption.send_keys(CAPTION)

# keep private for now
visibility = driver.find_element(By.CLASS_NAME, 'tiktok-select-selector').click()
private = driver.find_element(By.XPATH, '/html/body/div[3]/div/span[3]').click()

# post
post = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[2]/div[2]/div[9]/div[2]/button')
post.click()

In [217]:
import whisper

In [218]:
model = whisper.load_model('tiny')

100%|█████████████████████████████████████| 72.1M/72.1M [00:07<00:00, 9.97MiB/s]


In [221]:
result = model.transcribe('/Users/peter/Twitch/Twitch_Clips/xQc_2024-02-21_2069606012.mp4')

In [222]:
print(f' The text in video: \n {result["text"]}')

 The text in video: 
  gonna have shit singing peanut All right here's the situation at hand all right you want a Rob and all you want a rob and so that's fine, all right you want to come at us come at us Don't go puffing your chest out a fucking crisp and fucking a


In [219]:
vod_output_fn

'/Users/peter/Twitch/Twitch_VODs/xQc_2024-02-21_2069606012.mkv'

In [ ]:
## go to twitch channel --> videos --> filter by: clips --> train on chat in these clips

## potential model:
# language model: clean data, embed text data into vectors, and use cosine similarity to find word vectors of
# similar sentiment. Use twitch clips as training data, and use the twitch vods as test data?


## bag of words model --> simply calculates the amount of overlap between two documents --> Jaccard similarity
## NGRAM: is able to handle multi-word expressions, which BoW cannot do. Should combine Ngram w BoW together 
## Doc2Vec: represent documents as vectors and use the distance as a similarity measure
## Transformers: also work by representing words and sentences as vectors, but with a crucial difference that the vector representation 
## of a word is not fixed but is itself dependent on the context of the word.
## the state of the art in document similarity and other NLP tasks, but is very computationally expensive
## Term Frequency - Inverse Document Frequency (TF-IDF) is a widely used statistical method in natural 
## language processing and information retrieval. It measures how important a term is within a document 
## relative to a collection of documents (i.e., relative to a corpus).